You just got hired as the first and only data practitioner at a small business experiencing exponential growth. The company needs more structured processes, guidelines, and standards. Your first mission is to structure the human resources data. The data is currently scattered across teams and files and comes in various formats: Excel files, CSVs, JSON files...

You'll work with the following data in the `datasets` folder:
- __Office addresses__ are currently saved in `office_addresses.csv`. If the value for office is `NaN`, then the employee is remote. 
- __Employee addresses__ are saved on the first tab of `employee_information.xlsx`.
- __Employee emergency contacts__ are saved on the second tab of `employee_information.xlsx`; this tab is called `emergency_contacts`. However, this sheet was edited at some point, and the headers were removed! The HR manager let you know that they should be: `employee_id`, `last_name`, `first_name`, `emergency_contact`, `emergency_contact_number`, and `relationship`.
- __Employee roles, teams, and salaries__ have been exported from the company's human resources management system into a JSON file titled `employee_roles.json`. Here are the first few lines of that file:
```
{"A2R5H9":
  {
    "title": "CEO",
    "monthly_salary": "$4500",
    "team": "Leadership"
  },
 ...
}

## Import Datasets

### Libraries

In [125]:
import pandas as pd
import json

### Office addresses

In [126]:
office = pd.read_csv('datasets/office_addresses.csv')
office.head()

,office,office_country,office_city,office_street,office_street_number
0,Leuven Office,BE,Leuven,Martelarenlaan,38
1,ESB Office,US,New York City,Fifth Avenue,350
2,WeWork Office,GB,London,Old Street,207


### Employee addresses

In [127]:
emp_adress = pd.read_excel('datasets/employee_information.xlsx')
emp_adress.head()

,employee_id,employee_last_name,employee_first_name,employee_country,employee_city,employee_street,employee_street_number
0,A2R5H9,Hunman,Jax,BE,Leuven,Grote Markt,9
1,H8K0L6,Siff,Tara,GB,London,Baker Street,221
2,G4R7V0,Sagal,Gemma,US,New-York,Perry Street,66
3,M1Z7U9,Coates,Tig,FR,Paris,Rue de l'Université,7


### Employee emergency contacts

In [128]:
colnames_emerg_cont = ['employee_id','last_name','first_name','emergency_contact','emergency_contact_number','relationship']

In [129]:
emp_emerg_cont = pd.read_excel('datasets/employee_information.xlsx',sheet_name=1,header=None,names = colnames_emerg_cont)
emp_emerg_cont.head()

,employee_id,last_name,first_name,emergency_contact,emergency_contact_number,relationship
0,A2R5H9,Hunman,Jax,Opie Hurst,+32-456-5556-84,Brother
1,H8K0L6,Siff,Tara,Wendy de Matteo,+44-020-5554-333,Sister
2,G4R7V0,Sagal,Gemma,John Newmark,+1-202-555-194,Husband
3,M1Z7U9,Coates,Tig,Venus Noone,+1-202-555-0130,Wife


### Employee roles, teams and salaries

In [130]:
with open('datasets/employee_roles.json') as f:
   empl_roles = json.load(f)

In [131]:
empl_roles_df = pd.DataFrame(empl_roles) # to DataFrame
empl_roles_df = empl_roles_df.transpose() # transpose
empl_roles_df['employee_id'] = empl_roles_df.index
empl_roles_df.head()

,title,monthly_salary,team,employee_id
A2R5H9,CEO,$4500,Leadership,A2R5H9
H8K0L6,CFO,$4500,Leadership,H8K0L6
G4R7V0,Business Developer,$3000,Sales,G4R7V0
M1Z7U9,Office Manager,$2000,People Operations,M1Z7U9


## Merging

### By 'employee_id'

In [132]:
employees_final = pd.concat([empl_roles_df.set_index('employee_id'),
                    emp_emerg_cont.set_index('employee_id'),
                    emp_adress.set_index('employee_id')],axis = 1,join='inner')
employees_final['employee_id'] = employees_final.index
employees_final.head(20)

,title,monthly_salary,team,last_name,first_name,emergency_contact,emergency_contact_number,relationship,employee_last_name,employee_first_name,employee_country,employee_city,employee_street,employee_street_number,employee_id
employee_id,,,,,,,,,,,,,,,
A2R5H9,CEO,$4500,Leadership,Hunman,Jax,Opie Hurst,+32-456-5556-84,Brother,Hunman,Jax,BE,Leuven,Grote Markt,9,A2R5H9
H8K0L6,CFO,$4500,Leadership,Siff,Tara,Wendy de Matteo,+44-020-5554-333,Sister,Siff,Tara,GB,London,Baker Street,221,H8K0L6
G4R7V0,Business Developer,$3000,Sales,Sagal,Gemma,John Newmark,+1-202-555-194,Husband,Sagal,Gemma,US,New-York,Perry Street,66,G4R7V0
M1Z7U9,Office Manager,$2000,People Operations,Coates,Tig,Venus Noone,+1-202-555-0130,Wife,Coates,Tig,FR,Paris,Rue de l'Université,7,M1Z7U9


### By 'country'

In [133]:
employees_final = employees_final.merge(right=office,how='left',left_on='employee_country',
                           right_on='office_country',suffixes=('',''))
employees_final.head(20)

,title,monthly_salary,team,last_name,first_name,emergency_contact,emergency_contact_number,relationship,employee_last_name,employee_first_name,employee_country,employee_city,employee_street,employee_street_number,employee_id,office,office_country,office_city,office_street,office_street_number
0,CEO,$4500,Leadership,Hunman,Jax,Opie Hurst,+32-456-5556-84,Brother,Hunman,Jax,BE,Leuven,Grote Markt,9,A2R5H9,Leuven Office,BE,Leuven,Martelarenlaan,38.0
1,CFO,$4500,Leadership,Siff,Tara,Wendy de Matteo,+44-020-5554-333,Sister,Siff,Tara,GB,London,Baker Street,221,H8K0L6,WeWork Office,GB,London,Old Street,207.0
2,Business Developer,$3000,Sales,Sagal,Gemma,John Newmark,+1-202-555-194,Husband,Sagal,Gemma,US,New-York,Perry Street,66,G4R7V0,ESB Office,US,New York City,Fifth Avenue,350.0
3,Office Manager,$2000,People Operations,Coates,Tig,Venus Noone,+1-202-555-0130,Wife,Coates,Tig,FR,Paris,Rue de l'Université,7,M1Z7U9,NaN,NaN,NaN,NaN,NaN


In [134]:
employees_final.loc[:,employees_final.columns.str.startswith('office')] = employees_final.loc[:,employees_final.columns.str.startswith('office')].fillna('Remote') 
employees_final.head()

,title,monthly_salary,team,last_name,first_name,emergency_contact,emergency_contact_number,relationship,employee_last_name,employee_first_name,employee_country,employee_city,employee_street,employee_street_number,employee_id,office,office_country,office_city,office_street,office_street_number
0,CEO,$4500,Leadership,Hunman,Jax,Opie Hurst,+32-456-5556-84,Brother,Hunman,Jax,BE,Leuven,Grote Markt,9,A2R5H9,Leuven Office,BE,Leuven,Martelarenlaan,38.0
1,CFO,$4500,Leadership,Siff,Tara,Wendy de Matteo,+44-020-5554-333,Sister,Siff,Tara,GB,London,Baker Street,221,H8K0L6,WeWork Office,GB,London,Old Street,207.0
2,Business Developer,$3000,Sales,Sagal,Gemma,John Newmark,+1-202-555-194,Husband,Sagal,Gemma,US,New-York,Perry Street,66,G4R7V0,ESB Office,US,New York City,Fifth Avenue,350.0
3,Office Manager,$2000,People Operations,Coates,Tig,Venus Noone,+1-202-555-0130,Wife,Coates,Tig,FR,Paris,Rue de l'Université,7,M1Z7U9,Remote,Remote,Remote,Remote,Remote


### Select columns

In [135]:
employees_final.index = employees_final['employee_id']
employees_final = employees_final[['first_name','last_name','employee_country','employee_city', 'employee_street', 'employee_street_number', 'emergency_contact', 'emergency_contact_number', 'relationship', 'monthly_salary', 'team', 'title', 'office', 'office_country', 'office_city', 'office_street', 'office_street_number']]
employees_final.head()

,first_name,last_name,employee_country,employee_city,employee_street,employee_street_number,emergency_contact,emergency_contact_number,relationship,monthly_salary,team,title,office,office_country,office_city,office_street,office_street_number
employee_id,,,,,,,,,,,,,,,,,
A2R5H9,Jax,Hunman,BE,Leuven,Grote Markt,9,Opie Hurst,+32-456-5556-84,Brother,$4500,Leadership,CEO,Leuven Office,BE,Leuven,Martelarenlaan,38.0
H8K0L6,Tara,Siff,GB,London,Baker Street,221,Wendy de Matteo,+44-020-5554-333,Sister,$4500,Leadership,CFO,WeWork Office,GB,London,Old Street,207.0
G4R7V0,Gemma,Sagal,US,New-York,Perry Street,66,John Newmark,+1-202-555-194,Husband,$3000,Sales,Business Developer,ESB Office,US,New York City,Fifth Avenue,350.0
M1Z7U9,Tig,Coates,FR,Paris,Rue de l'Université,7,Venus Noone,+1-202-555-0130,Wife,$2000,People Operations,Office Manager,Remote,Remote,Remote,Remote,Remote
